# Geração Aumentada via Recuperação (RAG)

*Retrieval-Agumented Generation* (RAG) é uma técnica que consiste em "enriquecer" o contexto fornecido ao LLM antes do prompt propriamente dito, de forma que o modelo se detenha às informações apresentadas para gerar respostas.

Um exemplo de RAG em produção é o [NotebookLM](https://notebooklm.google), ferramenta do Google que permite analisar grandes quantidades de texto em arquivos.

## Arquitetura RAG

Em síntese, uma "busca" (*query*) é extraída do prompt, utilizada para fazer uma busca num banco de dados vetorial ("fontes de conhecimento") e de lá extrair informações relevantes para "enriquecer" o contexto.

Em seguida, o prompt com o contexto adicional recuperado é passado para o LLM, que gera a resposta final. A imagem a seguir demonstra essa arquitetura.

![A imagem apresenta um diagrama de fluxo que descreve um processo de interação com um modelo de linguagem de grande escala (LLM).](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/jumpstart/jumpstart-fm-rag.jpg)

[Fonte da imagem](https://aws.amazon.com/pt/what-is/retrieval-augmented-generation/)

## Fontes de conhecimento

Para cada tipo de aplicação, é desejável utilizar a arquitetura de banco de dados que melhor representa os dados do problema. Neste caso, em que o objetivo é recuperar informações via contexto, o melhor é utilizar um banco de dados vetorial, como o [ChromaDB](https://trychroma.com). Esses bancos de dados armazenam as informações na forma de vetores, de forma que vetores que descrevem temas correlacionados estão próximos (em termos da distância euclidiana) entre si.

In [6]:
from pathlib import Path
from utils import BLUE, GREEN, BOLD, RESET

datasource = list(Path("data/python-docs-text").glob("**/*.txt"))
print(f"{GREEN}Encontrei {len(datasource)} arquivos.{RESET}")

Encontrei 494 arquivos.


In [2]:
import chromadb

# Create a new database
client = chromadb.PersistentClient(path="chromadb")
coll = client.get_or_create_collection("python-docs")

In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
import tqdm

text_splitter = SemanticChunker(OpenAIEmbeddings())



files = []

for path in datasource:
    content = path.read_text()
    files.append(content)

print(f"Li {len(files)} arquivos.")

Li 494 arquivos.


## Salvando no Banco de Dados
A célula a seguir vai armazenar os documentos carregados no Chroma DB. Na minha máquina, para 494 arquivos, levou 17 min 46 s para importar 5350 documentos.

In [ ]:
count = coll.count()
print(f"Collection already contains {count} documents")


batch_size = 30
# Load the documents in batches of batch_size
for i in tqdm.tqdm(
    range(0, len(files), batch_size), desc="Adding documents", unit_scale=batch_size
):

    docs = text_splitter.create_documents(files[i : i + batch_size])
    ids = [f"id{i}-{j}" for j in range(len(docs))]
    coll.add(
        ids=ids,
        documents=[d.page_content for d in docs],
    )

new_count = coll.count()
print(f"Added {new_count - count} documents")

## Fazendo pesquisas na base de dados

Agora que já populamos a base de dados, podemos fazer perguntas especializadas.

In [ ]:
search = coll.query(
    query_texts=["interpretação de argumentos de linha de comando"],
    n_results=10,
)

In [ ]:
search["documents"][0]

['Se for... uma função definida por usuário:\n   O bloco de código da função é executado, passando-lhe a lista de\n   argumentos. A primeira coisa que o bloco de código fará é vincular\n   os parâmetros formais aos argumentos; isso é descrito na seção\n   Definições de função. Quando o bloco de código executa uma\n   instrução "return", isso especifica o valor de retorno da chamada\n   de função. um método embutido ou uma função embutida:\n   O resultado fica por conta do interpretador; veja Funções embutidas\n   para descrições de funções embutidas e métodos embutidos. um objeto classe:\n   Uma nova instância dessa classe é retornada. um método de instância de classe:\n   A função correspondente definida pelo usuário é chamada, com uma\n   lista de argumentos que é maior que a lista de argumentos da\n   chamada: a instância se torna o primeiro argumento. uma instância de classe:\n   A classe deve definir um método "__call__()"; o efeito é então o\n   mesmo como se esse método fosse ch

O resultado com a docs do Python não foi satisfatório.